# Circuit simulation examples

In [1]:
import obi_one as obi

In [2]:
circuit_path_prefix = "/Users/james/Documents/obi/additional_data/O1_data/O1_data/"
# circuit_path_prefix = "/Users/pokorny/Data/Circuits/"

### Loading two circuits

In [3]:
# circuit = obi.Circuit(name="ToyCircuit-S1-6k", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit = obi.Circuit(name="O1", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'O1' with 785632 neurons and 560631825 synapses


In [4]:
# circuit2 = obi.Circuit(name="ToyCircuit-S1-6k-duplicate", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
# circuit2 = obi.Circuit(name="nbS1-HEX0-beta", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit2 = obi.Circuit(name="O1_2", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes.size} neurons and {circuit2.sonata_circuit.edges.size} synapses")

Circuit 'O1_2' with 785632 neurons and 560631825 synapses


In [5]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(name="O1 Simulation", description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
sim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1")
stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=[10, 20])
rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=100)
property_neuron_set = obi.PropertyNeuronSet(
    property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": [2, 3], "synapse_class": ["INH"]}),
                     obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": [1, 2, 3], "synapse_class": ["EXC"]})],
)
sim_conf.add(sim_neuron_set, name='L1All')
sim_conf.add(stim_neuron_set, name='L1Stim')
sim_conf.add(rec_neuron_set, name='L1Rec')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, stim_duration=800, frequency=20, neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

<class 'obi_one.scientific.simulation.simulations.SimulationsForm'>


In [6]:
# simulations_form.model_dump(mode="json")

In [7]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

[2025-06-13 10:44:19,703] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-13 10:44:19,704] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-06-13 10:44:19,705] INFO: neuron_sets.L1Stim.random_sample: [10, 20]
[2025-06-13 10:44:19,705] INFO: neuron_sets.PropertyNeuronSet.property_filter: [layer=2,3,synapse_class=INH, layer=1,2,3,synapse_class=EXC]
[2025-06-13 10:44:19,706] INFO: initialize.circuit: [Circuit(type='Circuit', name='O1', path='/Users/james/Documents/obi/additional_data/O1_data/O1_data/circuit_config.json', matrix_path=None), Circuit(type='Circuit', name='O1_2', path='/Users/james/Documents/obi/additional_data/O1_data/O1_data/circuit_config.json', matrix_path=None)]
[2025-06-13 10:44:19,707] INFO: 
COORDINATE PARAMETERS
[2025-06-13 10:44:19,707] INFO: stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 10, neuron_sets.PropertyNeuronSet.property_filter: layer=2,3,synapse_class=INH, initialize.circuit: O1
[2025-06-13 10:44:19,708] INFO: stimuli.Current

{0: None,
 1: None,
 2: None,
 3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None,
 15: None}

In [8]:
# Deserialization
grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations/grid_scan/run_scan_config.json")